In [1]:
from pyspark.sql.functions import lower, regexp_replace, regexp_extract, col, trim, when, instr, lit, split, size, avg, isnan, when, count, isnull, mean, coalesce
from pyspark.sql.functions import concat_ws
from pyspark.sql import Window

In [2]:
from pyspark.sql import SparkSession

# Create a DataFrame using SparkSession
spark = (SparkSession
    .builder
    .appName("merging")
    .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/11 21:19:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# some_file.py
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '/home/jazzdung/E-Commerce-Support-System/script')

import lazada_data, shopee_data, model_data

23/02/11 21:19:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/02/11 21:19:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/02/11 21:19:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
lazada_df = lazada_data.get_full_data('/home/jazzdung/Downloads/product_lazada.ndjson', '/home/jazzdung/data/lazada_full.csv')
shopee_df = shopee_data.get_full_data('/home/jazzdung/Downloads/product_shopee.ndjson', '/home/jazzdung/data/shopee_full.csv')

lazada_df.printSchema()
shopee_df.printSchema()

Succeed!


Succeed!
root
 |-- product_name: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- price: integer (nullable = true)
 |-- brand: string (nullable = true)
 |-- num_review: integer (nullable = true)
 |-- attrs: string (nullable = true)
 |-- description: string (nullable = true)
 |-- url: string (nullable = true)
 |-- first_category: string (nullable = true)
 |-- second_category: string (nullable = false)
 |-- third_category: string (nullable = false)
 |-- shop_name: string (nullable = true)
 |-- shop_rating: double (nullable = true)
 |-- ship_on_time: double (nullable = true)
 |-- shop_reply_percectage: double (nullable = true)

root
 |-- attrs: string (nullable = true)
 |-- avg_rating: string (nullable = true)
 |-- num_review: integer (nullable = true)
 |-- num_sold: integer (nullable = true)
 |-- price: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- shipping: string (nullable = true)
 |-- url: string (nullable = true)
 |-- country: string

In [5]:
model_df = lazada_df.unionByName(shopee_df, allowMissingColumns=True)
model_df = model_data.grouping(model_df)
# model_df.count()
model_df.printSchema()
# model_df.show(1000, truncate=False)

root
 |-- product_name: string (nullable = true)
 |-- avg_rating: string (nullable = true)
 |-- price: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- num_review: integer (nullable = true)
 |-- attrs: string (nullable = true)
 |-- description: string (nullable = true)
 |-- url: string (nullable = true)
 |-- first_category: string (nullable = true)
 |-- second_category: string (nullable = false)
 |-- third_category: string (nullable = false)
 |-- shop_name: string (nullable = true)
 |-- shop_rating: double (nullable = true)
 |-- ship_on_time: double (nullable = true)
 |-- shop_reply_percectage: double (nullable = true)
 |-- num_sold: integer (nullable = true)
 |-- shipping: string (nullable = true)
 |-- country: string (nullable = true)
 |-- stock: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- shop_like_tier: string (nullable = true)
 |-- shop_num_review: integer (nullable = true)
 |-- shop_reply_percentage: double (nullable = true)
 |-- shop_rep

In [10]:
def get_model_data_all_col(shopee_df, lazada_df):

    # Load data
    df = shopee_df.unionByName(lazada_df, allowMissingColumns=True)

    df = model_data.fill_with_mean(df)
    df = model_data.drop_null_record(df)
    df = model_data.fill_with_blank(df)
    df = model_data.fill_with_no_info(df)
    df = model_data.concat_columns(df)
    df = model_data.drop_redundant_columns(df)

    df = model_data.grouping(df)
    
    return df

model_df_all_col = get_model_data_all_col(shopee_df, lazada_df)

(model_df_all_col  
    .coalesce(1)
    .write.option("header", True)
    .format("csv")
    .mode('overwrite')
    .csv('/home/jazzdung/data/model_all_col.csv'))
# model_df_func.count()
# model_df_func.show(100, truncate=False)
# model_df_func.select([c for c in model_df_func.columns if c not in {'product_name', 'description', 'url', 'attrs', 'name_description', 'augmented_description'}]).show(100, truncate=False)

AnalysisException: Cannot resolve column name "num_sold" among (product_name, avg_rating, price, brand, num_review, attrs, description, url, first_category, second_category, third_category, shop_name, shop_rating, ship_on_time, shop_reply_percectage)

In [16]:
def get_model_data_match_col(shopee_df, lazada_df):

    # Load data
    df = shopee_df.unionByName(lazada_df, allowMissingColumns=True)

    df = model_data.fill_with_mean(df)
    df = model_data.drop_null_record(df)
    df = model_data.fill_with_blank(df)
    df = model_data.fill_with_no_info(df)
    df = model_data.concat_columns(df)
    df = model_data.drop_redundant_columns(df)

    df = model_data.grouping(df)
    df = df.drop("avg_rating", "num_review", "shop_rating", "ship_on_time", "country", "num_sold", "shipping", "stock", "origin", "shop_like_tier", "shop_num_review", "shop_reply_time", "shop_creation_time", "shop_num_follower", "shop_reply_percentage")
    
    return df

model_df_match_col = get_model_data_match_col(shopee_df, lazada_df)

model_df_match_col.printSchema()
# model_df_match_col.show(100, truncate=False)

(model_df_match_col  
    .coalesce(1)
    .write.option("header", True)
    .format("csv")
    .mode('overwrite')
    .csv('/home/jazzdung/data/model_match_col.csv'))

root
 |-- price: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- first_category: string (nullable = false)
 |-- second_category: string (nullable = false)
 |-- third_category: string (nullable = false)
 |-- description: string (nullable = true)
 |-- shop_name: string (nullable = true)
 |-- shop_reply_percectage: double (nullable = true)
 |-- name_description: string (nullable = false)
 |-- augmented_description: string (nullable = false)

23/02/11 21:46:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/11 21:46:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/02/11 21:46:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/02/11 21:46:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
